# <center>[📊 Тюнинг гиперпараметров (**`Optuna`**)](https://stepik.org/lesson/825515/)</center>

<center> <img src='https://ucarecdn.com/fcc5e772-d63a-499d-8d94-b6c967867b35/' align="right" width="550" height="250" /> </center>
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🛠 Базовые концепции Optuna </a></li>

<li><a href="#c2">📂 Запуск на наших данных</a></li>
<li><a href="#с3">🖼 Визуализация </a>
<li><a href="#c4"> ✂️ Pruning</a>
<li><a href="#c5">🧠 Мысли и выводы</a>
<li><a href="#c5">🧸 Дополнительные материалы</a>

</li></ul></font></p>

<div class="alert alert-info">

* Подбирать параметры надо уметь!
* В предыдущих уроках мы рассматривали поиск оптимальных гиперпараметров классическими способами: полный перебор по сетке (`GridSearchCV`) и случайный перебор значений из заданного распределения (`Random Search`).
* Хотя `Random Search` уже значительно ускоряет процесс поиска, мы можем пропустить набор гиперпараметров, при котором модель показывает лучшее качество.<br>
* И тут, в голову может прийти идея: "А что, если нам немного поугадывать вначале, как в `Random Search`. А потом будем чаще проверять в тех местах, рядом с которыми модель показала лучшую точность?!". Такой метод называется - **Байесовский поиск гиперпараметров модели**.
* Самые популярные библиотеки, в которых реализован этот метод - `HyperOpt` и `Optuna`. (в нашей практике c `HyperOpt` часто случаются сбои и нестабильная работа, поэтому в этом ноутбуке сосредоточимся на **`Optuna`**)

    

# <center> Ключевые особенности фреймворка - **`Optuna`**
    
## Почему не `GridSearch`?

<div class="alert alert-info">
    
* 🎯 Легковесность и универсальность - можно подбирать оптимальные параметры под любые функции и метрики
* 🎁 SOTA алгоритмы, адаптированные для поиска гиперпараметров
* ⏱ Параллелизация и различные методы прунинга
* 📈 Встроенная визуализация
* 🤝 Интеграция со множеством популярных библиотек (бустинги, **sklearn**, **PyTorch**, **W&B** и другие)

Чтобы понять, как ее использовать, давайте разберем ее по частям.

In [1]:
!pip install optuna catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.7 MB/s eta 0:00:00


In [2]:
import optuna

import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, train_test_split

## В `Optuna` присутствуют 2 базовые сущности: <p id="c1"></p>    
### 1. `Study`: оптимизация, базирующаяся на `Objective` функции.

<!-- <div class="alert alert-info"> -->

В `Objective` функцию нужно написать код подсчета метрики, которую возвращаем. `Objective` вызывается Optun'ой много раз для подбора лучших параметров.
```python
def objective(trial, ...):
    # calculate score...
    return score
```

### 2. `Trial` - одно выполнение `Objective` функции

<!-- <div class="alert alert-info"> -->

В `trial` обьекте мы передаем параметры для "перебора", используя для каждого типа свой метод. К примеру

```python
# метод `suggest_float` показывает, что перебираем `float` значения, от 0 и до 1.5 границы.
param = trial.suggest_float('param', 0, 1.5)

# Категориальное значение
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Целочисленное значение
depth = trial.suggest_int('depth', 5, 8)

# Равномерное распределение
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```

### Study parameters

<div class="alert alert-info">

Инициализируем обьект `study`, который начнет перебор и сохранит в себе историю результатов.
Если мы стараемся увеличить метрику, а не уменьшить ошибку, то используем `create_study(direction='maximize')`
```python
study = optuna.create_study()
study.optimize(objective, n_trials=10)
```

<div class="alert alert-info">
    
В [`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) реализовано несколько методов (`sampler`) подбора параметров (в том числе классические):
* `GridSampler`
* `RandomSampler`
* `Tree-Structed Parzen Estimator` (`TPESampler` - самый популярный - дефолтный)
* `BruteForceSampler`
* И ещё [4 других](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers), также можно написать собственный сэмплер.

## Разберем небольшой пример и посмотрим как работает `Optuna`:
* Допустим у нас есть функция `y = (x+1)(x+5)(x-9)`, и мы хотим узнать `x`, при котором значение функции будет минимально.
* Создадим функцию `objective`, и передадим в аргументы объект `trial` - с помощью которого зададим интервал для `x` на `[-5, 5]`.


<center> <img src='../images/function4optuna.JPG' width="550">

In [3]:
# Ограничим логирование
optuna.logging.set_verbosity(30)

In [4]:
def objective(trial):
    x = trial.suggest_float("x", -8, 10)
    return (x + 1) * (x + 5) * (x - 9)


# создадим объект обучения, и запустим на 10 итераций; т.к мы ищем минимум, параметр direction оставляем дефолтным
study = optuna.create_study()

# запуск поиска
study.optimize(objective,
               n_jobs=-1,
               n_trials=250,
               show_progress_bar=True)

  0%|          | 0/250 [00:00<?, ?it/s]

In [5]:
optuna.visualization.plot_slice(study)

In [6]:
optuna.visualization.plot_optimization_history(study)

In [7]:
# выводим лучшие параметры
study.best_params

{'x': 5.162620587228549}

Видим результаты всех запусков и то, что минимум функции случился при `х = 5.16`, что близко к реальному минимуму.

## <center> 📦 Давайте запустим `Optuna` на наших данных </center> <p id="c2"></p>    

In [8]:
# загружаем датасет quickstart_train и отберем колонки
path = "../data/quickstart_train.csv"
train = pd.read_csv(path)

cat_features = ["model", "car_type", "fuel_type"]  # Выделяем категориальные признаки
targets = ["target_class", "target_reg"]
features2drop = ["car_id"]  # эти фичи будут удалены

# Отбираем итоговый набор признаков для использования моделью
filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

FileNotFoundError: [Errno 2] No such file or directory: '../data/quickstart_train.csv'

In [ ]:
CatBoostClassifier()

# 🌀 Советы по перебору параметров (очень логичные).
- Иметь понимание важности параметров
- Число `iterations` лучше взять с запасом и зафиксировать, при этом ограничив через `early_stopping_rounds`
- Подсмотреть/чувствовать диапазоны и шаг значений
- Исключить то, что перебирать не нужно. (`random_seed` , `eval_metric`, `thread_count` и прочее)
- Используйте информацию с прошлых попыток

Объявим функцию обучения `Catboost` с возращением предсказаний на `KFold` валидации

In [ ]:
def fit_catboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        'iterations' : 400, # Можно не перебирать, есть Early-Stopping
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 50),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.8),

        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 3, 9),

        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "used_ram_limit": "14gb",
        "eval_metric": "Accuracy", # Тоже стоит заранее определиться
    }


    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)

    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)


    clf = CatBoostClassifier(
        **param,
        thread_count=-1,
        random_seed=42,
        cat_features=cat_features,
    )

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
    )

    y_pred = clf.predict(X_val)
    return clf, y_pred

Напишем функцию `objective`, в которую поместим `KFold` валидацию, чтобы подбирать лучшие гиперпараметры на всем датасете

In [ ]:
from sklearn.metrics import accuracy_score


def objective(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X_train = train[filtered_features].drop(targets, axis=1, errors="ignore")
    y_train = train["target_class"]

    scores, models = [], []

    for train_idx, valid_idx in kf.split(X_train):
        train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_catboost(trial, train_data, valid_data) # Определили выше
        scores.append(accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break


    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

# 🚀 Запускаем Optuna!

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=600,
               n_jobs = -1,
               show_progress_bar=True,)

Посмотрим на лучшие параметры

In [ ]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Обучим итоговые модели уже на них

In [ ]:
valid_scores, models = objective(
    optuna.trial.FixedTrial(study.best_params),
    return_models=True,
)

In [ ]:
valid_scores, len(models)

# <center> 🖼 Визуализация оптимизаций </center>

<p id="c3"></p>    Чтобы посмотреть всю историю обучения, можно вывести ее в виде датафрейма

In [ ]:
trials_df = study.trials_dataframe().sort_values('value', ascending=False)
trials_df.head(3)

In [ ]:
# История изменения от числа испытаний
optuna.visualization.plot_optimization_history(study)

In [ ]:
# Зависимость в разрере по параметрам
params = ['l2_leaf_reg', 'colsample_bylevel', 'bagging_temperature', 'depth', 'bootstrap_type', 'subsample']
optuna.visualization.plot_slice(study,
                                params=params,
                                target_name = 'accuracy_score')

In [ ]:
# Важность параметров
optuna.visualization.plot_param_importances(study)

# <center> ✂️ Pruning - способ не идти по тупиковым траекториям перебора </center>  <p id="c4"></p>

<div class="alert alert-info">
    
В `Optuna` реализован параллельный поиск гиперпараметров - поиск стартует из нескольких рандомных мест и далее развивается по своей уникальной траектории.
`Pruning` — возможность останавливать поиск, когда обучающая кривая становится хуже прошлых лучших результатов, тем самым ускоряя процесс поиска.

<!-- ![Картинка](https://optuna.org/assets/img/pruning-example-with-caption.png) -->

<center> <img src='https://optuna.org/assets/img/pruning-example-with-caption.png' width="850">

<div class="alert alert-info">
    
Реализации прунеров есть для большинства известных ML фреймворков. Например, `callbacks` для бустингов:

* `CatBoost` : `optuna.integration.CatBoostPruningCallback`
* `XGBoost` : `optuna.integration.XGBoostPruningCallback`
* `LightGBM` : `optuna.integration.LightGBMPruningCallback`

<div class="alert alert-info">
    
### Основные виды прунеров:
* `Median Pruner` - самый популярный, каждые несколько итераций отбрасывает  половину процессов с наихудшим качеством
* `Successive Halving Algorithm` (SHA) - сначала запускаются `trials` с минимальными ресурсами (мало обучающих примеров, мало итераций), и на каждом следующем шаге отсекаем половину `trials` с худшим качеством + увеличиваем ресурсы
* `Percentile Pruner`, `Hyperband Pruner` и [другие](https://optuna.readthedocs.io/en/stable/reference/pruners.html)

<div class="alert alert-info">
    
Чтобы использовать прунинг в `CatBoost`, немного дополним функцию `objective`:

In [ ]:
from optuna.integration import CatBoostPruningCallback


def objective_catboost(trial):
    X = train[filtered_features].drop(targets, axis=1, errors="ignore")
    y = train["target_class"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    param = {
        'iterations' : 250,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 17),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),

        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 4, 9),

        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "used_ram_limit": "12gb",

        "eval_metric": "Accuracy",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    clf = CatBoostClassifier(
        **param, thread_count=-1, random_seed=42, cat_features=cat_features
    )

    # Создаем объект callback
    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_test, y_test),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
        callbacks=[pruning_callback],
    )  # Добавляем callback в fit

    # запускаем процесс прунинга
    pruning_callback.check_pruned()

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    return accuracy

<div class="alert alert-info">
    
Функция прунинга просто добавляется в функцию `create_study` в параметр `pruner`.
Воспользуемся `MedianPruner`.

In [ ]:
# study = optuna.create_study(
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
#     direction="maximize",
# )

# study.optimize(objective_catboost,
#                n_trials=1000,
#                n_jobs = -1,
#               )

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Поиск завершился заметно быстрее, при схожей точности.

# <center> 🧠 Советы и выводы

<div class="alert alert-info">
    
* 🧩 Подбор гиперпараметров, как правило не сильно улучшает качество моделей, но улучшает!
* 🎯 `Optuna` работает эффективнее перебора.
* 🧠 В `Optuna` есть способы не делать лишнюю работу.
* 🎓 Визуализация позволяет отбросить лишние параметры и сократить диапазоны значений.

# <center> 🧸 Дополнительные материалы
<p id="c5"></p>

<div class="alert alert-info">

* ⚠️ [Отличная статья](https://medium.com/@ali.soleymani.co/stop-using-grid-search-or-random-search-for-hyperparameter-tuning-c2468a2ff887) про методы поиска гиперпараметров и почему `Optuna` - топ!
* 🍏 У `Optuna` ещё множество полезных применений, вплоть до [препроцессинга датасетов](https://t.me/ds_private_sharing/60), ограничения только в вашей фантазии.
* 🍏 [Официальные туториалы](https://optuna.readthedocs.io/en/stable/tutorial/index.html)  с примерами.